In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

In [2]:
# load long video bench.
import pandas as pd

splits = {'validation': 'validation-00000-of-00001.parquet', 'test': 'test-00000-of-00001.parquet'}
df = pd.read_parquet("LongVideoBench/" + splits["validation"])

df

,video_id,id,video_path,subtitle_path,correct_choice,question,option0,option1,option2,option3,option4,duration,duration_group,question_category,topic_category,starting_timestamp_for_subtitles
0,86CxyhFV9MI,86CxyhFV9MI_0,86CxyhFV9MI.mp4,86CxyhFV9MI_en.json,1,"In the video, which subtitles appear at the sa...","promisc has come to an end, in and run away co...","run away countless times, i was just scared, i...","promisc has come to an end, in and front of ou...","promisc has come to an end, in and captain of ...",N/A,190.16,600,TOS,NP-News-Programs,0.0
1,86CxyhFV9MI,86CxyhFV9MI_1,86CxyhFV9MI.mp4,86CxyhFV9MI_en.json,2,Which subtitles appear at the same time as the...,[applause] and Seye Seye Seye Seye Seye,[applause] and my love that I made on the ke is,"captain of the godson, three three three three...","[applause] and captain of the godson, three th...",N/A,190.16,600,TOS,NP-News-Programs,0.0
2,9PD3ciudpIE,9PD3ciudpIE_0,9PD3ciudpIE.mp4,9PD3ciudpIE_en.json,0,"In front of an ancient building, a woman walks...",yellow,green,blue,red,N/A,467.92,600,T2A,NP-News-Programs,0.0
3,9PD3ciudpIE,9PD3ciudpIE_1,9PD3ciudpIE.mp4,9PD3ciudpIE_en.json,0,"In front of a light yellow wall, there is a wo...",red short sleeves,red long sleeves,black long sleeves,pink short sleeves,N/A,467.92,600,T2A,NP-News-Programs,0.0
4,5qMcDQd17Y4,5qMcDQd17Y4_0,5qMcDQd17Y4.mp4,5qMcDQd17Y4_en.json,3,"In the scene, there is a man wearing a white s...",cell phone,car,computer,street light,N/A,584.64,600,T2O,KS-Knowledge-STEM,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,GGwHSz9towk,GGwHSz9towk_1,GGwHSz9towk.mp4,GGwHSz9towk_en.json,3,"The man with curly hair wearing a blue shirt, ...",Petting,Taking a photo,Painting,Observing,N/A,193.09,600,S2E,KA-Knowledge-Art,0.0
1333,_EUDpS9UF9o,_EUDpS9UF9o_0,_EUDpS9UF9o.mp4,_EUDpS9UF9o_en.json,0,"In the scene, there's a person cutting a green...","On the table, there's also a silver bowl conta...",Oven,Bread,Watch,N/A,9.00,15,T2O,LC-Lifestyle-Cooking-Recipes,1267.0
1334,7XWqI121-Q4,7XWqI121-Q4_0,7XWqI121-Q4.mp4,7XWqI121-Q4_en.json,2,In a bedroom with two warm-colored table lamps...,Changed into a black coat,Visited a museum,Took a shower,Wrote something in front of the computer,Put on shoes,954.46,3600,E3E,LV-Lifestyle-Life-Vlogs,0.0
1335,7XWqI121-Q4,7XWqI121-Q4_1,7XWqI121-Q4.mp4,7XWqI121-Q4_en.json,3,A person wearing khaki-colored pants with a le...,The woman opens a laptop on the bed,The woman visits a museum,The woman sits on a bed in the bedroom and talks,The woman stands on the bridge and takes a sel...,The woman goes to eat in the dining hall,954.46,3600,E3E,LV-Lifestyle-Life-Vlogs,0.0


In [3]:
import numpy as np

"""
r is the region of the slice
list_of_feature is video -> images -> features, rows is frame's features
"""
def svd_reduce_dim(list_of_feature, r = 10):

    u, s, v = np.linalg.svd(list_of_feature, full_matrices=False)
    X = u
    output = X[:, :r]

    # print(output.shape)
     
    # a matrix can directly to use maxvol
    return output

# define max vol
"""
piv is select of key_frames index. 
initail_M is svd output.
"""
from maxvolpy.maxvolpy.maxvol import rect_maxvol

def max_vol_extract_index(initail_M, tol, minK=1, maxK=500):    
    piv, C = rect_maxvol(initail_M, tol)
    return piv

# define consine
"""
cosine similarity -> reduce same frames after max vol
filtered_indices is key frames with no similarity
"""
def filter_similar_frames(features, selected_indices, similarity_threshold=0.9):
    filtered_indices = [selected_indices[0]]
    for i in range(1, len(selected_indices)):
        previous_feature = features[filtered_indices[-1]]
        current_feature = features[selected_indices[i]]
        similarity = 1 - cosine(previous_feature, current_feature)
        
        if similarity < similarity_threshold:
            filtered_indices.append(selected_indices[i])
    
    return filtered_indices

In [4]:
from torch.utils.data import Dataset, DataLoader
import pyarrow.parquet as pq
import math
import pysubs2
import os
import json

def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]

class VideoMMEDataset(Dataset):
    video_formats = ['.mp4', '.avi', '.mov', '.mkv']

    def __init__(self, video_folder, subtitle_folder, data_list):
        self.video_folder = video_folder
        self.subtitle_folder = subtitle_folder
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        line = self.data_list[idx]

        video_path = line['video_path']
        
        for fmt in self.video_formats:
            temp_path = os.path.join(self.video_folder, video_path)
            if os.path.exists(temp_path):
                video_path = temp_path
                break

        subtitle_path = os.path.join(self.subtitle_folder, line['subtitle_path'])
        
        try:
            video_tensor = {"type": "video", "video": video_path, "fps": 1}
        except:
            traceback.print_exc()
            print(f'It occurs error when reading {video_ytid}')
            video_tensor = None
        
        if video_tensor is not None and os.path.exists(subtitle_path):
            
            with open(subtitle_path, 'r') as file:
                data = json.load(file)
            
            subtitles = [item["line"] for item in data if "line" in item and item["line"]]
            
            subtitles = ' '.join(subtitles)
            
        else:
            subtitles = ""
        
        return {
            'video': video_tensor,
            'subtitle': subtitles,
            'record': line,
        }

def load_parquet(parquet_file):
    table = pq.read_table(parquet_file)
    df = table.to_pandas()
    
    jsons = []
    for record in df.itertuples():
        if len(jsons) < int(5340):
            jsons.append({
                "video_id": record.video_id,
                "id": record.id,
                "video_path": record.video_path,
                "subtitle_path": record.subtitle_path,
                "duration": record.duration,
                "topic_category": record.topic_category,
                "question_category": record.question_category,
                "questions": [
                    {
                        "question": record.question,
                        "option0": record.option0,
                        "option1": record.option1,
                        "option2": record.option2,
                        "option3": record.option3,
                        "option4": record.option4,
                        "correct_choice": record.correct_choice,
                    }
                ]
            })
    return jsons

def collate_fn(batch):
    vid = [x['video'] for x in batch]
    sub = [x['subtitle'] for x in batch]
    rcs = [x['record'] for x in batch]
    return vid, sub, rcs

def videomme_dump(record, instruct, output):
    letters = ['A', 'B', 'C', 'D']
    pred_answer = re.findall('[\(\ \[]*([A-D])[\)\.\ \]]*', output)
    try:
        assert len(pred_answer) >= 1, 'The video \"{}\" output \"{}\" is not in the expected format'.format(record['youtube_id'], instruct + '\n' + output)
        pred_answer = pred_answer[0].strip()
        pred_answer = pred_answer.strip('()')
        pred_idx = letters.index(pred_answer)
    except:
        traceback.print_exc()
        pred_idx = 2
    return letters[pred_idx]

def build_videomme_eval(args, processor):
    questions = load_parquet(args.question_file)
    questions = get_chunk(questions, args.num_chunks, args.chunk_idx)
    dataset = VideoMMEDataset(args.video_folder, args.subtitle_folder, questions, processor)
    dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_fn)
    return dataloader

def reduce_video_frames1(video_input, max_frames=768):
    """
    Reduce the number of frames in a video tensor to a maximum of max_frames.
    
    Args:
    video_input (torch.Tensor): Input video tensor of shape [num_frames, channels, height, width]
    max_frames (int): Maximum number of frames in the output tensor
    
    Returns:
    torch.Tensor: Video tensor with reduced number of frames
    """
    num_frames, channels, height, width = video_input.shape
    
    if num_frames <= max_frames:
        return video_input
    
    keep_indices = torch.linspace(0, num_frames - 1, max_frames).long()
    
    reduced_video = video_input[keep_indices]
    
    return reduced_video

def reduce_video_frames(vision_model, vision_processor, frames, frame_ts_id_group, batch_size):
    device = vision_model.device
    all_feats = []
    num_frames = len(frames)

    with torch.no_grad():
        for i in range(0, num_frames, batch_size):
            batch_images = frames[i:i + batch_size]  # PIL list
            inputs = vision_processor(images=batch_images, return_tensors="pt", padding=True).to(device)
            feats = vision_model.get_image_features(**inputs)
            all_feats.append(feats.cpu())

    image_features = torch.cat(all_feats, dim=0)   # [num_frames, hidden_dim]

    low_dimensions = svd_reduce_dim(image_features, r=8)
    low_dimensions_np = low_dimensions.cpu().numpy()
    selections = max_vol_extract_index(low_dimensions_np, tol=0.3)
    selections.sort()
    
    if len(selections) > 1 and len(selections) % 2 != 0:
        selections = selections[:-1]

    selected_indices = selections
    selected_frames = [frames[idx] for idx in selected_indices]
    selected_ts_group = [frame_ts_id_group[idx] for idx in selected_indices]

    return selected_frames, selected_ts_group, selected_indices

In [5]:
import sys
import argparse

sys.argv = ['ipykernel_launcher.py', 
            '--model-path', 'Qwen/Qwen2-VL-7B-Instruct',
            '--video-folder', 'LongVideoBench/videos',
            '--subtitle-folder', 'LongVideoBench/subtitles',
            '--question-file', 'LongVideoBench/validation-00000-of-00001.parquet',
            '--answer-file', 'WACV/answer-normal180f-72B-fps2-new.json']

parser = argparse.ArgumentParser()
parser.add_argument('--model-path', required=True)
parser.add_argument('--video-folder', required=True)
parser.add_argument('--subtitle-folder', required=True)
parser.add_argument('--question-file', required=True)
parser.add_argument('--answer-file', required=True)
parser.add_argument("--num-chunks", type=int, default=1)
parser.add_argument("--chunk-idx", type=int, default=0)
parser.add_argument("--device", type=str, required=False, default='cuda:1')
parser.add_argument("--batch-size", type=int, default=4)
parser.add_argument("--num-workers", type=int, default=8)
args = parser.parse_args()

In [6]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from decord import VideoReader, cpu    # pip install decord
from scipy.spatial import cKDTree
import numpy as np
import math

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-4_5', trust_remote_code=True,  # or openbmb/MiniCPM-o-2_6
    attn_implementation='sdpa', torch_dtype=torch.bfloat16, device_map="auto") # sdpa or flash_attention_2, no eager
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-4_5', trust_remote_code=True)  # or openbmb/MiniCPM-o-2_6

/home/user/conda/envs/MaxInfo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


In [7]:
def build_videomme_eval(args):
    questions = load_parquet(args.question_file)
    questions = get_chunk(questions, args.num_chunks, args.chunk_idx)
    dataset = VideoMMEDataset(args.video_folder, args.subtitle_folder, questions)
    dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_fn)
    return dataloader

val_loader = build_videomme_eval(args)

In [8]:
from transformers import AutoProcessor, CLIPModel

vision_model_path = "openai/clip-vit-large-patch14-336"
vision_model = CLIPModel.from_pretrained(vision_model_path, device_map="auto")
vision_processor = AutoProcessor.from_pretrained(vision_model_path)

/home/user/conda/envs/MaxInfo/lib/python3.10/site-packages/accelerate/utils/modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['logit_scale']
  warnings.warn(
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 7752.87it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 8065.97it/s]


### Basic Model read video

In [22]:
MAX_NUM_FRAMES=128 # Indicates the maximum number of frames received after the videos are packed. The actual maximum number of valid frames is MAX_NUM_FRAMES * MAX_NUM_PACKING.
MAX_NUM_PACKING=1  # indicates the maximum packing number of video frames. valid range: 1-6
TIME_SCALE = 0.1 

def map_to_nearest_scale(values, scale):
    tree = cKDTree(np.asarray(scale)[:, None])
    _, indices = tree.query(np.asarray(values)[:, None])
    return np.asarray(scale)[indices]


def group_array(arr, size):
    return [arr[i:i+size] for i in range(0, len(arr), size)]

def encode_video(video_path, choose_fps=3, force_packing=None):
    def uniform_sample(l, n):
        gap = len(l) / n
        idxs = [int(i * gap + gap / 2) for i in range(n)]
        return [l[i] for i in idxs]
    vr = VideoReader(video_path, ctx=cpu(0))
    fps = vr.get_avg_fps()
    video_duration = len(vr) / fps
        
    if choose_fps * int(video_duration) <= MAX_NUM_FRAMES:
        packing_nums = 1
        choose_frames = round(min(choose_fps, round(fps)) * min(MAX_NUM_FRAMES, video_duration))
        
    else:
        packing_nums = math.ceil(video_duration * choose_fps / MAX_NUM_FRAMES)
        if packing_nums <= MAX_NUM_PACKING:
            choose_frames = round(video_duration * choose_fps)
        else:
            choose_frames = round(MAX_NUM_FRAMES * MAX_NUM_PACKING)
            packing_nums = MAX_NUM_PACKING

    frame_idx = [i for i in range(0, len(vr))]      
    frame_idx =  np.array(uniform_sample(frame_idx, choose_frames))

    if force_packing:
        packing_nums = min(force_packing, MAX_NUM_PACKING)
    
    frames = vr.get_batch(frame_idx).asnumpy()

    frame_idx_ts = frame_idx / fps
    scale = np.arange(0, video_duration, TIME_SCALE)

    frame_ts_id = map_to_nearest_scale(frame_idx_ts, scale) / TIME_SCALE
    frame_ts_id = frame_ts_id.astype(np.int32)

    assert len(frames) == len(frame_ts_id)

    frames = [Image.fromarray(v.astype('uint8')).convert('RGB') for v in frames]
    frame_ts_id_group = group_array(frame_ts_id, packing_nums)
    
    return frames, frame_ts_id_group

In [23]:
from tqdm import tqdm
import torch
import copy
import json
from scipy.spatial.distance import cosine

results = []

for i, (videos, subtitles, records) in enumerate(tqdm(val_loader)):
    torch.cuda.empty_cache()
    
    for b in range(len(videos)):
        video = videos[b]
        subtitle = subtitles[b]
        record = records[b]

        new_record = copy.deepcopy(record)
        new_record_sub = copy.deepcopy(record)

        if video is None:
            new_record['missing'] = True
            new_record_sub['missing'] = True
            results.append({
                'record': new_record,
                'record_sub': new_record_sub,
                'answer': None,
                'converted_choice': None
            })
            continue
        else:
            new_record['missing'] = False
            new_record_sub['missing'] = False

        questions = record['questions']
        q = questions[0]['question']

        ops = []
        for i, letter in enumerate(['A', 'B', 'C', 'D', 'E']):
            opt = questions[0].get(f'option{i}', None)
            if opt:
                ops.append(f"{letter}. {opt}")

        instruct = "Select the best answer to the following multiple-choice question based on the video. Respond with only the letter (A, B, C, D, or E) of the correct option.\n"
        instruct += f"{q}\n"
        for op in ops:
            instruct += f"{op}\n"
        instruct += "The best answer is: "
        instruct = f"This video's subtitles are listed below:\n{subtitle}\n" + instruct

        video_path = video['video']
        fps = 1
        force_packing = None
        frames, frame_ts_id_group = encode_video(video_path, fps, force_packing=force_packing)
        
        frames, frame_ts_id_group, selected_indices = reduce_video_frames(vision_model, vision_processor, frames, frame_ts_id_group, batch_size = 16)

        assert len(frames) == len(frame_ts_id_group), f"Length mismatch: {len(selected_frames)} frames vs {len(selected_ts_group)} ts"

        msgs = [{'role': 'user', 'content': frames + [instruct]}]

        answer = model.chat(
            msgs=msgs,
            tokenizer=tokenizer,
            use_image_id=False,
            max_slice_nums=1,
            temporal_ids=frame_ts_id_group
        )

        choice_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
        converted_choice = choice_mapping.get(questions[0]['correct_choice'], None)

        # 保存结果
        results.append({
            'record': new_record,
            'record_sub': new_record_sub,
            'answer': answer,
            'num frame': len(frames),
            'converted_choice': converted_choice
        })
        print("=" * 30)
        print("Model answer:", answer)
        print("Correct choice:", converted_choice)
        print("Num Frames:", len(frames))
        print("=" * 30)

with open("evaluation_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("All results saved to evaluation_results.json")

  0%|          | 0/335 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Model answer: C
Correct choice: B
Num Frames: 62
Model answer: D
Correct choice: C
Num Frames: 62
Model answer: A
Correct choice: A
Num Frames: 70


  0%|          | 1/335 [00:53<4:55:24, 53.07s/it]

Model answer: D
Correct choice: A
Num Frames: 70
Model answer: D
Correct choice: D
Num Frames: 74
Model answer: A
Correct choice: A
Num Frames: 74
Model answer: C. speaking
Correct choice: C
Num Frames: 70


  1%|          | 2/335 [01:46<4:55:26, 53.23s/it]

...

100%|█████████▉| 334/335 [4:09:09<00:51, 51.44s/it]

Model answer: D
Correct choice: D
Num Frames: 72


100%|██████████| 335/335 [4:09:30<00:00, 44.69s/it]

Model answer: A
Correct choice: A
Num Frames: 72


All results saved to evaluation_results.json
